## Setup & load data

In [1]:
import pandas as pd
import plotly.express as px

In [ ]:
sale_df = pd.read_csv(r"03_Cleaned Dataset\sales_clean.csv")
prop_df =pd.read_csv(r"03_Cleaned Dataset\properties_clean.csv")
address_df = pd.read_csv(r"\03_Cleaned Dataset\addresses_clean.csv")
agent_df = pd.read_csv(r"\03_Cleaned Dataset\agent_clean.csv")
client_df = pd.read_csv(r"\03_Cleaned Dataset\client_clean.csv")

In [3]:
saledetails_df = sale_df.merge(prop_df,on="property_id",how="left")
saledetails_df = saledetails_df.merge(address_df, on="address_id", how="left")
saledetails_df = saledetails_df.merge(agent_df, on="agent_id", how="left")
saledetails_df = saledetails_df.merge(client_df, on="client_id", how="left")
saledetails_df.head()

,sale_id,property_id,client_id,agent_id,owner_id_x,sale_date,sale_amount,commission_rate,commission_amount,commission_payment_method,...,gender_x,age_x,email,phone,hire_date,agent_type,name_y,gender_y,address_id_y,age_y
0,1,662,679,107,1745,2023-05-16,886392.38,0.05,44319.62,Cash,...,Female,55,stephanieking@gmail.com,936-982-2586x686,2024-01-09,Sales Agent,Theresa Thompson,Female,2447,27
1,2,1422,2057,207,548,2022-04-07,237485.78,0.02,4749.72,Bank Transfer,...,Male,27,douglasryanphd@gmail.com,+1-690-611-0111x0914,2023-11-15,Sales Agent,Donna Mullen,Female,358,30
2,3,873,201,86,515,2023-04-19,235812.08,0.05,11790.60,Cash,...,Male,41,amandacarlson@hotmail.com,9028323398,2021-04-04,Sales Agent,Robert Lindsey,Male,1971,36
3,4,552,573,51,373,2024-10-02,266874.04,0.03,8006.22,Cash,...,Male,35,daltonrichardson@hotmail.com,343.360.2041,2020-02-12,Sales Agent,Lauren Morales,Female,2863,57
4,5,2020,1062,91,929,2023-11-18,156286.92,0.02,3125.74,Bank Transfer,...,Female,38,kristinstewart@gmail.com,615-943-8407x596,2024-09-21,Sales Agent,Angel Ryan,Female,950,40


In [4]:
saledetails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sale_id                    1645 non-null   int64  
 1   property_id                1645 non-null   int64  
 2   client_id                  1645 non-null   int64  
 3   agent_id                   1645 non-null   int64  
 4   owner_id_x                 1645 non-null   int64  
 5   sale_date                  1645 non-null   object 
 6   sale_amount                1645 non-null   float64
 7   commission_rate            1645 non-null   float64
 8   commission_amount          1645 non-null   float64
 9   commission_payment_method  1645 non-null   object 
 10  commission_payment_date    1645 non-null   object 
 11  owner_id_y                 1645 non-null   int64  
 12  property_manager_id        1645 non-null   int64  
 13  address_id_x               1645 non-null   int64

In [5]:
saledetails_df['sale_date']=pd.to_datetime(saledetails_df['sale_date'])
saledetails_df['commission_payment_date']=pd.to_datetime(saledetails_df['commission_payment_date'])
saledetails_df.rename(columns={"address_id_x": "property_address_id",'name_x':'saleagent_name','gender_x':'saleagent_gender','age_x':'saleagent_age','name_y':'customer_name','gender_y':'customer_gender', 'address_id_y':'customer_address', 'age_y':'customer_age'}, inplace=True)

In [6]:
print(saledetails_df.columns)


Index(['sale_id', 'property_id', 'client_id', 'agent_id', 'owner_id_x',
       'sale_date', 'sale_amount', 'commission_rate', 'commission_amount',
       'commission_payment_method', 'commission_payment_date', 'owner_id_y',
       'property_manager_id', 'property_address_id', 'listing_date',
       'property_type', 'rooms', 'bathrooms', 'parking', 'condition',
       'listing_type', 'listing_price', 'street', 'city', 'state', 'latitude',
       'longitude', 'saleagent_name', 'saleagent_gender', 'saleagent_age',
       'email', 'phone', 'hire_date', 'agent_type', 'customer_name',
       'customer_gender', 'customer_address', 'customer_age'],
      dtype='object')


# Sale Breakdown Analysis

to helps businesse see patterns, growth areas, and weaknesses.


In [7]:
total_sales = saledetails_df["sale_amount"].sum()
print(total_sales)

767652807.5699999


Instead of only saying "We made $768M this year, answer:

1. How much from each estate?

In [8]:
city_sales = saledetails_df.groupby(["state","city"])['sale_amount'].sum().reset_index()
city_sales["percentage"] = (city_sales["sale_amount"] / total_sales) * 100

city_sales.head()

,state,city,sale_amount,percentage
0,CA,Fresno,24528140.43,3.195213
1,CA,Long Beach,13675189.76,1.781429
2,CA,Los Angeles,19902770.59,2.592679
3,CA,Oakland,22188449.39,2.890428
4,CA,Sacramento,27032858.20,3.521495


In [9]:
state_fig = px.histogram(city_sales,x="city",y="sale_amount",title="Sales by state  and city",color="state")
state_fig.show()

2. How much and How many from each agent?

In [10]:
agent_sales = saledetails_df.groupby(["saleagent_name"]).agg(total_sales= ('sale_amount',"sum"),
                                                   count_Sales= ('sale_id',"count")).reset_index()

agent_sales.head()

,saleagent_name,total_sales,count_Sales
0,Alison Fisher,7792704.84,20
1,Allison Arellano,5778318.08,17
2,Amanda Carlson,8576463.04,21
3,Amanda Randall,5906359.47,13
4,Amber Brown,12264129.90,20


In [11]:
top_agents = agent_sales.sort_values(by="total_sales", ascending=False).head(10)

agent_fig = px.bar(top_agents,
                   x="saleagent_name",
                   y="total_sales",
                   text="total_sales",
                   color="total_sales",
                   color_continuous_scale="Blues",
                title="Top 10 Agents by Total Sales")

agent_fig.update_layout(
    xaxis_title="Agent",
    yaxis_title="Total Sales"
)
agent_fig.show()

3. How much by customer age categories? 

In [12]:
age_bins = [0,25,45,65,saledetails_df['customer_age'].max()]
labels = ['Young', 'Adult', 'Middle-aged', 'Senior']
saledetails_df["customer_age_category"]=pd.cut(saledetails_df["customer_age"],bins=age_bins,labels=labels,right=False)

saledetails_df["customer_age_category"].value_counts()

customer_age_category
Adult          938
Middle-aged    528
Young          148
Senior          24
Name: count, dtype: int64

In [13]:
sales_byage = saledetails_df.groupby('customer_age_category')['sale_amount'].sum().reset_index()
sales_byage.head()

C:\Users\Computec\AppData\Local\Temp\ipykernel_6788\635054154.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,customer_age_category,sale_amount
0,Young,6.888030e+07
1,Adult,4.293260e+08
2,Middle-aged,2.527070e+08
3,Senior,1.274480e+07


In [14]:
fig = px.pie(
    sales_byage,
    names='customer_age_category',
    values='sale_amount',            # قيمة كل slice: إجمالي المبيعات
    title='Total Sales Distribution by Customer Age Category',
    hole=0.5
)
fig.show()

# Trend Analysis

 How much from each time period (month/quarter/year)?

In [15]:
saledetails_df['sales_year']=saledetails_df['sale_date'].dt.year
saledetails_df['sales_month']=saledetails_df['sale_date'].dt.month

In [16]:
yearly_sales = saledetails_df.groupby(['sales_year']).agg(total_sales= ('sale_amount',"sum"),
                                                   count_Sales= ('sale_id',"count")).reset_index()
yearly_sales.head()

,sales_year,total_sales,count_Sales
0,2020,1.688191e+07,33
1,2021,5.065747e+07,119
2,2022,1.070954e+08,227
3,2023,1.853660e+08,407
4,2024,4.076521e+08,859


In [17]:
fig = px.line(
    yearly_sales,
    x='sales_year',
    y=['total_sales','count_Sales'],
    markers=True,
    title="Yearly Sales: Total Amount and Number of Transactions"
)

fig.update_yaxes(type="log", title_text="Value")

fig.update_xaxes(title_text="Year")

fig.show()

In [18]:
monthly_sales = saledetails_df.groupby(['sales_month']).agg(total_sales= ('sale_amount',"sum"),
                                                   count_Sales= ('sale_id',"count")).reset_index()
monthly_sales.head()

,sales_month,total_sales,count_Sales
0,1,43864606.66,98
1,2,46995427.52,88
2,3,43525407.12,92
3,4,56448670.55,131
4,5,60960720.94,130


In [27]:
# Moving Average 
monthly_sales['total_sales_smooth'] = monthly_sales['total_sales'].rolling(window=3, min_periods=1).mean()


In [28]:
fig = px.line(monthly_sales,
              x='sales_month',
              y=['total_sales', 'total_sales_smooth'],
              title="Monthly Sales with 3-Month Moving Average",
              labels={'value': 'total_sales', 'sale_date': 'sales_month'},
              color_discrete_sequence=['#636EFA', '#EF553B'])

fig.show()